In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Training set
ts_i = pd.read_csv('./Data/features_inter.csv')
ts_u = pd.read_csv('./Data/features_union.csv')
ts_i = ts_i.drop(['parcelid'],axis=1)
ts_u = ts_u.drop(['parcelid'],axis=1)
ts_i.head()

,logerror,finishedsquarefeet12,lotsizesquarefeet,taxamount,longitude,latitude,regionidzip,yearbuilt,month
0,0.0276,1684.0,7528.0,6735.880000,-118.488536,34.280990,96370.0,1959.0,1
1,-0.1684,2263.0,3643.0,10153.020000,-117.677556,33.668120,96962.0,2014.0,1
2,-0.0040,2217.0,11423.0,5984.582647,-118.175032,34.136312,96293.0,1940.0,1
3,0.0218,839.0,70859.0,3048.740000,-118.309000,33.755800,96222.0,1987.0,1
4,-0.0050,2283.0,6000.0,5488.960000,-117.700234,33.485643,96961.0,1981.0,1


In [3]:
def mape_loss(t,p):
    sum = 0
    for i in range(t.shape[0]):
        sum += abs(t[i]-p[i])
    return sum/t.shape[0]
def metric_rf(X_train, y_train, X_test, y_test, model, cri):
    print(model)
    fit_time = 0
    score_time = 0
    test_score = 0
    train_score = 0
    result = cross_validate(model, X_test, y_test, scoring=cri)
    for i in range(3):
        fit_time += result['fit_time'][i]
        score_time += result['score_time'][i]
        test_score += result['test_score'][i]
        train_score += result['train_score'][i]
    pred = model.predict(X_test)
    m = mape_loss(y_test, pred)
    return [fit_time/3, score_time/3, test_score/3, train_score/3, m]
# def mapeobj(preds,dtrain):
#     gaps = dtrain.get_label()
#     grad = np.sign(preds-gaps)/gaps
#     hess = 1/gaps
#     grad[(gaps==0)] = 0
#     hess[(gaps==0)] = 0
#     return grad,hess  

# def evalmape(preds, dtrain):
#     gaps = dtrain.get_label()
#     err = abs(gaps-preds)/gaps
#     err[(gaps==0)] = 0
#     err = np.mean(err)
#     return 'error',err 

In [5]:
# Random Forest with MSE
ts_list = [ts_i,ts_u]
model_list = []
frames = []
for m in ts_list:
    for i in range(5):
        for j in range(5):
            print("Round %d:" % (i*5+j+1))
            
            # Training set
            ts = m
            y = ts[['logerror']]
            X = ts.drop('logerror',axis=1)
            
            # Split data with 0.75
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
            print (X_train.shape)
            print (y_train.shape)
            print (X_test.shape)
            print (y_test.shape)
            
            rf = RandomForestRegressor(n_estimators=5*(i+1), criterion='mse', max_depth=5*(j+1), random_state=10)
            rf.fit(X_train, y_train)
            model_list.append(rf)
            metric = metric_rf(X_train, np.array(y_train), X_test, np.array(y_test), rf, 'neg_mean_squared_error')
            print(metric)
            result = pd.DataFrame.from_items([(i*5+j,metric)],\
                                             orient='index',\
                                             columns=['fit_time','score_time','test_score','train_score','MAPE'])
            frames.append(result)
            print("****************************")
m = pd.concat(frames)
print(m)

Round 1:
(115563, 8)
(115563, 1)
(49528, 8)
(49528, 1)


/Users/CJL-RMBP/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
           oob_score=False, random_state=10, verbose=0, warm_start=False)


/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[0.44882567723592121, 0.0064798196156819659, -0.0039515122363833538, -0.0038895296569295824, array([ 0.04304814])]
****************************
Round 2:
(115563, 8)
(115563, 1)
(49528, 8)
(49528, 1)


...

[7.2579356829325361, 0.090252876281738281, -0.0040128381019200248, -0.0018338032228019522, array([ 0.04360122])]
****************************
Round 25:
(115563, 11)
(115563, 1)
(49528, 11)
(49528, 1)


/Users/CJL-RMBP/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
           oob_score=False, random_state=10, verbose=0, warm_start=False)


/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[7.912638982137044, 0.11299626032511394, -0.0041240514799873208, -0.0012266518656330648, array([ 0.04420262])]
****************************
    fit_time  score_time  test_score  train_score               MAPE
0   0.448826    0.006480   -0.003952    -0.003890  [0.0430481369619]
1   0.735590    0.007675   -0.004026    -0.003521  [0.0430202102912]
2   1.170298    0.014719   -0.004186    -0.002898  [0.0439766635863]
3   1.626233    0.017258   -0.004457    -0.002035  [0.0457152991138]
4   2.135770    0.047136   -0.004671    -0.001630  [0.0476606502488]
5   1.135545    0.008839   -0.003907    -0.003838  [0.0428598045899]
6   1.715810    0.018245   -0.003986    -0.003540  [0.0431251551606]
7   2.844340    0.029232   -0.004053    -0.002715  [0.0434616925624]
8   2.891359    0.036966   -0.004209    -0.001888  [0.0446667348555]
9   3.431405    0.047638   -0.004342    -0.001402  [0.0457053169226]
10  1.184193    0.014558   -0.003838    -0.003773  [0.0425468563759]
11  1.862509    0.018484   -0.00

In [6]:
m.sort_values(by='MAPE')

,fit_time,score_time,test_score,train_score,MAPE
11,1.862509,0.018484,-0.003849,-0.003391,[0.0424719830356]
21,3.758887,0.028926,-0.003864,-0.003417,[0.0424820845041]
10,1.184193,0.014558,-0.003838,-0.003773,[0.0425468563759]
15,1.296632,0.011756,-0.003889,-0.003823,[0.0427361342855]
21,3.215697,0.031380,-0.003888,-0.003419,[0.0427393485371]
16,2.396214,0.024857,-0.003903,-0.003449,[0.0427626202411]
12,3.345938,0.055578,-0.003915,-0.002659,[0.0428111946215]
10,1.203683,0.010408,-0.003902,-0.003840,[0.0428224822496]
22,4.187831,0.050625,-0.003945,-0.002630,[0.0428519674689]
5,1.135545,0.008839,-0.003907,-0.003838,[0.0428598045899]


In [17]:
best = model_list[11]
y = ts_i[['logerror']]
X = ts_i.drop('logerror',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
pred = best.predict(X_test)
error = pred - y_test

ValueError: Shape of passed values is (49528, 49528), indices imply (1, 49528)

In [ ]:
pickle.dump(rf_best, open('random_forest.pkl','wb'))
my_rf = pickle.load(open('random_forest.pkl','rb'))
my_rf

In [ ]:
pred = rf.predict(X_train)

In [ ]:
plt.figure("Log error scatter")
plt.scatter(range(temp.shape[0]), np.sort(temp.values))
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('True Value')
ax.set_ylabel('Prediction')
plt.show()